In [2]:
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import OneHotEncoder

from keras.layers import Input, Dense, Activation
from keras.models import Sequential
from keras.models import Model

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

print(tf.__version__)

1.11.0


In [63]:
#the first thing we need to do is get the data and the labels for the data. 
DataFrameMain = pd.read_csv("C:/Users/tsmar/election_dataframe.csv")
DataFrame = DataFrameMain.select_dtypes(include=["int", "float"])
headers = list(DataFrame.columns.values)
jj = int(0.8*len(DataFrame))
kk = int(0.2*len(DataFrame))

#these are the columns I will train on for now.
DataFrame["meanGOP"] = DataFrame[["Republicans 2016", "Republicans 2012", "Republicans 2008"]].mean(axis=1) / 100
DataFrame["meanDEM"] = DataFrame[["Democrats 2016", "Democrats 2012", "Democrats 2008"]].mean(axis=1) / 100
DataFrame["Percent of adults with a bachelor's degree or higher, 2012-2016"] = DataFrame["Percent of adults with a bachelor's degree or higher, 2012-2016"] / 100
DataFrame["Percent of adults completing some college or associate's degree, 2012-2016"] = DataFrame["Percent of adults completing some college or associate's degree, 2012-2016"] / 100
DataFrame["Percent of adults with a high school diploma only, 2012-2016"] = DataFrame["Percent of adults with a high school diploma only, 2012-2016"] / 100
DataFrame["Percent of adults with less than a high school diploma, 2012-2016"] = DataFrame["Percent of adults with less than a high school diploma, 2012-2016"]/100
#make test and train data split 80/20
trainData = DataFrame[["meanGOP","meanDEM","Percent of adults with a bachelor's degree or higher, 2012-2016","Percent of adults completing some college or associate's degree, 2012-2016","Percent of adults with a high school diploma only, 2012-2016","Percent of adults with less than a high school diploma, 2012-2016"]][0:jj]
testData = DataFrame[["meanGOP","meanDEM","Percent of adults with a bachelor's degree or higher, 2012-2016","Percent of adults completing some college or associate's degree, 2012-2016","Percent of adults with a high school diploma only, 2012-2016","Percent of adults with less than a high school diploma, 2012-2016"]][-1*kk:]
trainLabels = []
testLabels = []
#build the train Labels
for index in range(0, len(trainData["meanGOP"])):
    rawData = trainData.values
    j = rawData[index][0]
    k = rawData[index][1]
    if j > k:
        trainLabels.append(np.array([1,0])) #gop
    else:
        trainLabels.append(np.array([0,1])) #dem
    
#build the test labels
for index in range(0, len(testData["meanGOP"])):
    rawData = testData.values
    j = rawData[index][0]
    k = rawData[index][1]
    if j > k:
        testLabels.append(np.array([1,0])) #gop
    else:
        testLabels.append(np.array([0,1])) #dem

#for now a sequential model should be useful, the input shape is only 4 because we only have 4 input classes.
model = Sequential([
    Dense(16, input_dim=4, kernel_initializer='normal'),
    Activation('relu'),
    Dense(16, kernel_initializer='normal'),
    Activation('relu'),
    Dense(2, kernel_initializer='normal'),
    Activation('softmax')
])

#build the model with appropriate values
model.compile(optimizer='rmsprop', #for binary classification
              loss='binary_crossentropy', #same as above
              metrics=['accuracy']) #how close is our ouput to what it was supposed to be?
#print a summary of the model
model.summary()
data = np.array(trainData[["Percent of adults with a bachelor's degree or higher, 2012-2016","Percent of adults completing some college or associate's degree, 2012-2016","Percent of adults with a high school diploma only, 2012-2016","Percent of adults with less than a high school diploma, 2012-2016"]].values, dtype='float64')
test = np.array(testData[["Percent of adults with a bachelor's degree or higher, 2012-2016","Percent of adults completing some college or associate's degree, 2012-2016","Percent of adults with a high school diploma only, 2012-2016","Percent of adults with less than a high school diploma, 2012-2016"]].values, dtype='float64') 

#we learn from data, and we expect output equivalent to the labels. 
model.fit(data, np.array(trainLabels, dtype='float64'), epochs=5, verbose=2, batch_size=128)
scores = model.evaluate(test, np.array(testLabels, dtype='float64')[-1*kk:])

#get accuracy
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("input = ",data[0], " output = ", trainLabels[0])


C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_166 (Dense)            (None, 16)                80        
_________________________________________________________________
activation_166 (Activation)  (None, 16)                0         
_________________________________________________________________
dense_167 (Dense)            (None, 16)                272       
_________________________________________________________________
activation_167 (Activation)  (None, 16)                0         
_________________________________________________________________
dense_168 (Dense)            (None, 2)                 34        
_________________________________________________________________
activation_168 (Activation)  (None, 2)                 0         
Total params: 386
Trainable params: 386
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
 -